# Step19 ~ 21 Variableをnumpy.arrayライクに使えるようにする

## インスタンス変数とメソッドを作成する

### インスタンス変数

まずは現状のVariableクラスの使い勝手を確認しましょう。<br>
numpy.arrayとshapeと同じ情報を取得してみます。

In [ ]:
import numpy as np
from Core import Variable

x = Variable(np.array(([1.0, 2.0], [3.0, 4.0])))

↓のセルにxに格納された配列のshapeを取り出してみましょう。

In [ ]:
# 期待値：(2, 2)

これでもshapeの情報を取得できるのですが、numpy.arrayよりひと手間多くなります。<br>
そこでnumpy.arrayと同じように取得できるようにVariableクラスを改良しましょう。

具体的には**インスタンス変数shapeを追加**します。<br>
以下をCore.pyのVariableクラスの中に追加します。

インスタンス変数を定義する場合は@propertyを付けます。

In [ ]:
@property
def shape(self):
    return self.data.shape

すると以下ようにアクセスできるようになります。

In [ ]:
import numpy as np
from Core import Variable

x = Variable(np.array(([1.0, 2.0], [3.0, 4.0])))

x.shape

次は**dtype**を追加してみます。以下をVariableクラスに追加します。

In [ ]:
@property
def dtype(self):
    return self.data.dtype

以下のようにアクセスできるようになります。

In [ ]:
import numpy as np
from Core import Variable

x = Variable(np.array(([1.0, 2.0], [3.0, 4.0])))

x.dtype

ここで@propertyについて説明します。<br>
以下の変数**data**の書き方でもインスタンス変数の定義出来ました。<br>

In [ ]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError("{} is not supported".format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

両者の違いは更新の可否です。
shapeは更新できませんが、nameは更新可能です。

In [ ]:
x.shape = (1, 3)

In [ ]:
x.data = np.array(1.0)
x.data

これは@propertyがオブジェクト指向プログラミングで言うgetterに対応しているからです。<br>
setterが無ければgetterで取得できる値を更新させることはできません。

`@プロパティ`は**デコレータ**と呼ばれます。<br>
デコレータはメソッドやクラスに特定の機能を追加するために使用します。

shapeは参照のみが可能なインスタンス変数としたいので@propertyによって実装します。<br>

*ちなみに参照渡しなのかと思って確認してみたら一見すると矛盾する結果が返ってきて不明です。*

In [ ]:
# idが違うので値渡し？
print(id(x.shape))
print(id(x.data.shape))

# 比較するとTrueなので参照渡し？
isEqualId = id(x.shape) == id(x.data.shape)
print(isEqualId)

### 組込み関数

Pythonの組込み関数の使い勝手を変えることもできます。<br>
そのためには組込み関数を使う際に呼び出される特殊メソッドをオーバーライドします。
今回はprint関数の出力をいじってみましょう。

まずは現在のprintを確認しましょう。

In [ ]:
x = Variable(np.array(([1.0, 2.0], [3.0, 4.0])))
print(x)

Variableクラスのインスタンスであることが出力されます。<br>
printもnumpy.arrayのような出力が得られるように改良しましょう。

print関数を実行すると、引数に渡したオブジェクトの特殊メソッド`__repr__`が呼び出されます。<br>
このメソッドをオーバーロードして使いやすくしましょう。
以下をVariableクラスに追加します。

In [ ]:
def __repr__(self):
    if self.data is None:
        return "variable(None)"
    p = str(self.data).replace("\n", "\n" + " " * 9)
    return "variable(" + p + ")"

In [ ]:
import numpy as np
from Core import Variable

x = Variable(np.array(([1.0, 2.0], [3.0, 4.0])))

print(x)

### Step19演習

In [ ]:
"""
以下を実装してください。
1. ndim ← numpy.arrayの同名メソッドと同じ値を出力する。

2. name ← Variableクラスのインスタンスの名前を格納する。デフォルトはNone。更新可とする。

3. size ← numpy.arrayの同名メソッドと同じ値を出力する。

4. len ← len関数をVariableに対して使用した場合、len関数をVariable.dataに対して使用した結果を返す。
   ※len関数に対応する特殊メソッドは__len__

"""

import numpy as np
from Core import Variable

data = np.array(([1.0, 2.0], [3.0, 4.0]))
x = Variable(data)


print("# 1.ndim #########################")
print(f"想定: {data.ndim}")

# print(x.ndim)


print("\n\n# 2.name #########################")
name = "x"
# x = Variable(np.array(([1.0, 2.0],[3.0,4.0])), name)
print(f"想定: {name}")

# print(x.name)


print("\n\n# 3.size #########################")
print(f"想定: {data.size}")

# print(x.size)


print("\n\n# 4.len ##########################")
print(f"想定: {len(data)}")

# print(len(x))

## 演算子のオーバーロード

### Variable同士の計算

次に演算についてもnumpy.arrayライクに動かせるようにします。<br>
まずは掛け算について考えてみましょう。

掛け算レイヤの順伝播と逆伝播のイメージは以下です：<br>
![MulClass](./fig/MulClass.png)

これをクラスとして実装すると以下になります。

Core.pyにコピーして動作確認します。

In [ ]:
# 乗算
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


# 関数として呼び出せるようにしておく
def mul(x0, x1):
    return Mul()(x0, x1)

まずは動作確認

In [ ]:
from Core import mul

x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = mul(x, y)
print(z)
z.backward()
print(x.grad, y.grad)

この関数でVariableの*演算子をオーバーロードします。
以下をVariable内にコピーします。

In [ ]:
# *演算子のオーバーロード
def __mul__(self, other):
    return mul(self, other)

ちなみにmulメソッドには以下のように入力が渡されます。

![mul](./fig/mul.png)

先ほどmulを直接呼び出したところを*演算子に書き換えて実行してみましょう。

In [ ]:
import numpy as np
from Core import Variable, mul

x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = x * y
z = mul(x, y)
print(z)
z.backward()
print(x.grad, y.grad)

また、Pythonではオブジェクトを代入するが出来るので以下のようにオーバーロードすることもできます。

In [ ]:
Variable.__mul__ = mul

これで掛け算を自然な書き方で実装できるようになりました。

### VariableとNumpyのデータ型の計算

次はVariableとNumpyのデータ型との計算も自然な書き方で実装できるようにしましょう。

現状ではVariable同士でなければうまくいきません。

In [ ]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

# Variable同士
print(x * y)

# Variableとnumpy.array
print(x * np.array(3.0))

ということは（numpy.arrayなどの）Variable以外のデータ型が入力されたときは裏側でVariableに変換すればうまく計算できそうです。

この方針でVariableを改良します。

まずはnumpy.arrayに対応させます。
Variable型に変換する処理は以下の関数を使います。

In [ ]:
def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

これをCore.pyにコピーして動作確認します。

In [ ]:
import numpy as np
from Core import Variable, as_variable

x = Variable(np.array(2.0))
print(as_variable(x))

y = np.array(2.0)
print(as_variable(y))

Variable型に変換出来ていることが分かります。

この関数をFunctionに組み込みましょう。<br>
__call__メソッドの冒頭で入力をVariable型に統一するようにします。

In [ ]:
def __call__(self, *inputs):
    inputs = [as_variable(x) for x in inputs]  # ←これを追加

    xs = [x.data for x in inputs]
    ys = self.forward(*xs)

ではnumpy.arrayも計算できるようになったことを確認しましょう。<br>
さきほどエラーになった計算をもう一度実行します。

In [ ]:
import numpy as np
from Core import Variable

x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

# Variable同士
print(x * y)

# Variableとnumpy.array
print(x * np.array(3.0))

これでnumpy.arrayとは計算できるようになりました。<br>

### Variableとfloatやintのデータ型の計算

次はintやfloatと計算できるようにします。

現状ではどうなるのか確認しましょう。

In [ ]:
import numpy as np
from Core import Variable

x = Variable(np.array(2.0))
y = 3.0

# Variableとfloat
print(x * y)

これを処理できるように実装を変更します。<br>
実は既に作成した関数を使って対応することが出来ます。

numpy.arrayまでは対応できたので、floatがnumpy.arrayに変換できれば計算できるようになります。<br>
そこでas_array関数を使いnumpy.arrayに変換します。<br>
mulの入力にas_array関数を適用してからMulクラスに渡すようにします。

In [ ]:
def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)

計算できるようになったことを確認します。

In [ ]:
import numpy as np
from Core import Variable

x = Variable(np.array(2.0))
y = 3.0

# Variableとfloat
print(x * y)

Variableと他の型が自然に計算できるようになりました。

が、実はまだ不完全です。<br>
問題は掛け算の左項と右項を入れ替えたときに起こります。


In [ ]:
# 左項と右項を入れ替えた場合
print(y * x)

floatとVariableが*演算子をサポートしていないというエラーになります。

なぜこのようなエラーになるのか順番に見ていきます。

この計算を処理する際、以下の流れで処理しようとします。

(1). まず*演算子の左項の__mul__メソッドが呼ばれます。<br>
が、floatの__mul__メソッドを見ると実装がありません。

In [ ]:
# floatの__mul__メソッド
def __mul__(self, __x: float) -> float:
    ...  # ...は任意処理を表現する特殊な定数

(2). 次にVariableのメソッドが呼ばれます。<br>
右項にある場合、__mul__ではなく__rmul__メソッドが呼ばれます。<br>
このメソッドはVariableに未実装なので結果として掛け算を処理できずエラーになっているのです。

以上の考察からVariableに__rmul__メソッドを実装すればうまくいきそうです。<br>
※floatの__mul__を実装することもできるでしょうが、Variableクラスの掛け算を改良しているのでVariableの__rmul__を実装する方が無難です。<br>
そもそも組込み型の実装を変えることは影響範囲が膨大になるので避けるべきかと思います。

__rmul__は自身（self）が右項、相手（other）が左項です。<br>
![rmul](./fig/rmul.png)

掛け算は交換法則が成り立つので左項と右項の区別を付けなくても問題ありません。<br>
そこでmulを__rmul__にそのまま使用します。

以下をCore.py内のVariableクラスの定義以降にコピーしましょう。

In [ ]:
Variable.__rmul__ = mul

では不具合の解消を確認してみましょう。

In [ ]:
import numpy as np
from Core import Variable

x = Variable(np.array(2.0))
y = 3.0

print(y * x)

最後に左項がnumpy.arrayのときの問題点を解消してこのステップを終わりにします。

現状では掛け算の順序によって計算結果が異なります。

In [ ]:
x = Variable(np.array([2.0]))
y = np.array([3.0])

In [ ]:
x * y

In [ ]:
y * x

これは呼び出される演算子の優先度が原因です。<br>
先述の通り*演算子の処理では先に左項の__mul__メソッドが呼ばれるため、どちらを左項に置くかで結果が異なります。<br>

これを解消するためにはVariableの優先度を上げてnumpy.arrayの__mul__ではなくVariableの__rmul__が呼ばれるようにします。<br>
優先度は__array_priority__を指定します。デフォルトは0.0なので整数値に指定すればVariableのメソッドを優先するようになります。

以下をVariableクラス内に追加しましょう。

In [ ]:
__array_priority__ = 1.0

In [ ]:
import numpy as np
from Core import Variable

x = Variable(np.array([2.0]))
y = np.array([3.0])
y * x

### Step20・21演習

In [ ]:
"""
演習1

以下の計算ができるようにしてください。
* 以前のステップで作成した加算用のクラスを使用して構いません。
* +演算子の特殊メソッドは__add__です。
"""
import numpy as np
from Core import Variable

# a = Variable(np.array(2.0))
# b = Variable(np.array(3.0))
# c = np.array(4.0)

# d = c + (a + b)
# print(d)  # 期待値:Variable(9.0)

"""
演習2

mul関数では2つ目の引数x1にのみas_array関数を適用しています。
以下の計算では左項がfloat型なのでエラーになりそうですが、問題なく実行されます。
1つ目の引数x0にas_array関数を適用しなくてよい理由を説明してください。

# mul関数の実装 #################
def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)
"""

x = Variable(np.array(2.0))
y = 3.0

print(y * x)